In [62]:
PATH = '../data/empregos/ocupacao_H_M_2022-2023.csv'

from sktime.forecasting.naive import NaiveForecaster
from sktime.split import temporal_train_test_split
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.sarimax import SARIMAX
from datetime import datetime
import pandas as pd
import numpy as np
import time

df = pd.read_csv(PATH)
df.set_index('Ano', inplace=True)
df.index = pd.to_datetime(df.index, format='%Y')
df.index = df.index.to_period('M')

PATH1 = '../data/empregos/pessoas_ocupadas_2020-2023.csv'
df1 = pd.read_csv(PATH1)
df1 = df1.T
df1 = df1.rename(columns={0:"populacao"})
df1.index = pd.to_datetime(df1.index)
df1.index = df1.index.to_period("M")
df1.populacao = pd.to_numeric(df1.populacao)
df1.populacao *= 1000


In [63]:
df

,Total,Homens,Mulheres
Ano,,,
2022-01,95208,54853,40355
2023-01,99205,56860,42345


In [68]:
man_2023 = df.Homens['2023-01']
woman_2023 = df.Mulheres['2023-01']
total_2023 = df.Total['2023-01']

percent_man = man_2023 / total_2023 
percent_woman = 1 - percent_man
print(percent_man, percent_woman)

0.5731565949296911 0.42684340507030893


In [69]:
pop_homens = df1
pop_mulheres = df1
pop_homens = round(pop_homens.populacao * percent_man)
pop_mulheres = round(pop_mulheres.populacao * percent_woman)

In [74]:
print(pop_homens.head())
print(pop_mulheres.head())

2020-01    53369476.0
2020-02    51662043.0
2020-03    49700701.0
2020-04    48174385.0
2020-05    47414379.0
Freq: M, Name: populacao, dtype: float64
2020-01    39745524.0
2020-02    38473957.0
2020-03    37013299.0
2020-04    35876615.0
2020-05    35310621.0
Freq: M, Name: populacao, dtype: float64


In [75]:
homens = pd.DataFrame(pop_homens)
mulheres = pd.DataFrame(pop_mulheres)

In [82]:
QUANTIDADE_DE_MESES_PARA_PREVER=12
def model_arima(df):
    y_train, y_test = temporal_train_test_split(df)
    fhh = np.arange(len(y_test)+QUANTIDADE_DE_MESES_PARA_PREVER)+1
    model = AutoARIMA(sp=12)
    model.fit(y_train)
    y_arima = model.predict(fh=fhh)
    return y_arima

In [83]:
y_arimaH = model_arima(homens)
y_arimaM = model_arima(mulheres)

/home/liputer/Documentos/pibiti/app-live-data/notebooks/.venv/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/home/liputer/Documentos/pibiti/app-live-data/notebooks/.venv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/liputer/Documentos/pibiti/app-live-data/notebooks/.venv/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/home/liputer/Documentos/pibiti/app-live-da

In [84]:
y_arimaH = round(y_arimaH)
y_arimaM = round(y_arimaM)
y_arimaH.to_csv("predict_ocupacao_homens_2024-2025.csv")
y_arimaM.to_csv("predict_ocupacao_mulheres_2024-2025.csv")